# Data generator v1

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

## Sort key function

In [2]:
def sort_object(name):
    tmp = name.split("_")
    return int(tmp[0])

In [3]:
def sort_overlay(name):
    return int(name[:-4])

## Overlay function

In [4]:
def overlay_transparent(background, overlay, x, y):

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background

# make overlay img

In [ ]:
random.seed(13)

object_img_list = os.listdir("./object")
object_img_list = sorted(object_img_list, key=sort_object)
background_img_list = os.listdir("./background")

cnt = 0

# print(object_img_list)
for one_img in object_img_list:
    class_num = one_img.split("_")[1][0]
    
    # one object img
    object_img = cv2.imread("./object/" + one_img, cv2.IMREAD_UNCHANGED)
    object_img_h = object_img.shape[0]
    object_img_w = object_img.shape[1]
    
    # w/h : ratio * h = w, h/w : 1 / ratio * w = h
    object_ratio = float(object_img_w) / float(object_img_h)

    background_img_sample = random.sample(background_img_list, 40)

    # generate 5 overlay image
    for one_background_img in background_img_sample:
        # label txt file open
        label_txt_file = open("./label/" + str(cnt) + ".txt",'w')
        
        background_img = cv2.imread("./background/" + one_background_img)
        background_img_h = background_img.shape[0]
        background_img_w = background_img.shape[1]
        
        if object_ratio > 1: # w > h
            resize_object_w = random.randrange(int(background_img_w * 0.4), int(background_img_w * 0.7))
            resize_object_h = int(resize_object_w * (1 / object_ratio))
            
        else:
            resize_object_h = random.randrange(int(background_img_h * 0.4), int(background_img_h * 0.7))
            resize_object_w = int(resize_object_h * object_ratio)
        
        # random object (x,y)
        random_x = random.randrange(0, int(background_img_w * 0.7))
        random_y = random.randrange(0, int(background_img_h * 0.7))
        
        #
        if random_x + resize_object_w > background_img_w:
            resize_object_w = background_img_w - random_x
        
        if random_y + resize_object_h > background_img_h:
            resize_object_h = background_img_h - random_y
        
        
        label_w_ratio = float(resize_object_w) / float(background_img_w)
        label_h_ratio = float(resize_object_h) / float(background_img_h)
        
#         label_w_ratio = float(resize_object_w) / float(background_img_w)
#         label_h_ratio = float(resize_object_h) / float(background_img_h)
        
        # label x ratio & y ratio
        label_x_ratio = float(random_x + resize_object_w/2) / float(background_img_w)
        label_y_ratio = float(random_y + resize_object_h/2) / float(background_img_h)
        
        # object resize and overlay to background
        resize_object_img = cv2.resize(object_img,(resize_object_w,resize_object_h))
        overlay_img = overlay_transparent(background_img, resize_object_img, random_x, random_y)
        
        # label txt write
        label_txt_file.write(str(class_num)+" "+\
                             str(label_x_ratio)+" "+\
                             str(label_y_ratio)+" "+\
                             str(label_w_ratio)+" "+\
                             str(label_h_ratio)+"\n")
        label_txt_file.close()
        
        cv2.imwrite("./overlay/"+str(cnt)+".jpg", overlay_img)
        
#         print(random_x, random_y)
#         print(resize_object_w, resize_object_h)
        
        cnt += 1
        

## check labeling

In [ ]:
class_txt_file = open("class.txt","r")
class_label = class_txt_file.readlines()

overlay_img_list = os.listdir("./overlay")
overlay_img_list = sorted(overlay_img_list, key=sort_overlay)

for one_overlay_img in overlay_img_list:
    
    overlay_img = cv2.imread("./overlay/"+one_overlay_img)
    
    overlay_img_h = overlay_img.shape[0]
    overlay_img_w = overlay_img.shape[1]
    
    txt_file = open("./label/" + one_overlay_img[:-4] + ".txt","r")
    labels = txt_file.readlines()
    
    for one_label in labels:
        split_label = one_label.split(" ")
        
        class_name = class_label[int(split_label[0])][:-1]
        x_ratio = float(split_label[1])
        y_ratio = float(split_label[2])
        w_ratio = float(split_label[3])
        h_ratio = float(split_label[4][:-1])
        
        x = int(x_ratio * overlay_img_w)
        y = int(y_ratio * overlay_img_h)
        w = int(w_ratio * overlay_img_w)
        h = int(h_ratio * overlay_img_h)
        
        cv2.rectangle(overlay_img, (x-w/2,y-h/2), (x+w/2,y+h/2), (255,0,0), 3)
    
    cv2.imwrite("./check/" + one_overlay_img, overlay_img)

## make dataset v2

In [ ]:
data_size = 2000

random.seed(13)

object_img_list = os.listdir("./object_v4")
object_img_list = sorted(object_img_list, key=sort_object)
background_img_list = os.listdir("./real_background")

background_img_sample = random.sample(background_img_list, data_size)

cnt = 0

# generate overlay image
for one_background_img in background_img_sample:
    # label txt file open
    label_txt_file = open("./label/" + str(cnt) + ".txt",'w')

    background_img = cv2.imread("./real_background/" + one_background_img)
    background_img_h = background_img.shape[0]
    background_img_w = background_img.shape[1]
    
    
    random_object_num = random.randrange(1, 6)
    random_object_list = random.sample(object_img_list, random_object_num)
    #print(random_object_num)
    for one_object_img in random_object_list:
        class_num = one_object_img.split("_")[1][0]
        
        object_img = cv2.imread("./object_v4/" + one_object_img, cv2.IMREAD_UNCHANGED)
        object_img_h = object_img.shape[0]
        object_img_w = object_img.shape[1]
        
        object_ratio = float(object_img_w) / float(object_img_h)
        
    
        if object_ratio > 1: # w > h
            resize_object_w = random.randrange(int(background_img_w * 0.4), int(background_img_w * 0.7))
            resize_object_h = int(resize_object_w * (1 / object_ratio))

        else:
            resize_object_h = random.randrange(int(background_img_h * 0.4), int(background_img_h * 0.7))
            resize_object_w = int(resize_object_h * object_ratio)

        # random object (x,y)
        random_x = random.randrange(0, int(background_img_w * 0.7))
        random_y = random.randrange(0, int(background_img_h * 0.7))

        if random_x + resize_object_w > background_img_w:
            resize_object_w = background_img_w - random_x

        if random_y + resize_object_h > background_img_h:
            resize_object_h = background_img_h - random_y


        label_w_ratio = float(resize_object_w) / float(background_img_w)
        label_h_ratio = float(resize_object_h) / float(background_img_h)

#         label_w_ratio = float(resize_object_w) / float(background_img_w)
#         label_h_ratio = float(resize_object_h) / float(background_img_h)

        # label x ratio & y ratio
        label_x_ratio = float(random_x + resize_object_w/2) / float(background_img_w)
        label_y_ratio = float(random_y + resize_object_h/2) / float(background_img_h)
        
        #print(resize_object_w, resize_object_h)
        # object resize and overlay to background
        resize_object_img = cv2.resize(object_img,(resize_object_w,resize_object_h))
        background_img = overlay_transparent(background_img, resize_object_img, random_x, random_y)

        # label txt write
        label_txt_file.write(str(class_num)+" "+\
                             str(label_x_ratio)+" "+\
                             str(label_y_ratio)+" "+\
                             str(label_w_ratio)+" "+\
                             str(label_h_ratio)+"\n")
    
    cv2.imwrite("./overlay/"+str(cnt)+".jpg", background_img)
    label_txt_file.close()
    cnt += 1


## make dataset v3

In [16]:
############# parameter ################
data_size = 8000

random.seed(13)

object_path = "./object_v4"
background_path = "./real_background"
label_path = "./label_v1"
overlay_path = "./overlay_v1"

max_object_num = 8
########################################

object_folder_list = os.listdir(object_path)
object_folder_list = sorted(object_folder_list, key=sort_object)

background_img_list = os.listdir(background_path)

background_img_sample = int(data_size / len(background_img_list) + 1) * background_img_list
print(len(background_img_sample))

cnt = 0

# generate overlay image
for one_background_img in background_img_sample:
    # label txt file open
    label_txt_file = open(label_path +"/" + str(cnt) + ".txt",'w')

    background_img = cv2.imread(background_path + "/" + one_background_img)
    background_img_h = background_img.shape[0]
    background_img_w = background_img.shape[1]
    
    random_object_num = random.randrange(1, max_object_num)
    random_object_list = random.sample(object_folder_list, random_object_num)
    
    #print(random_object_num)
    for one_object_folder in random_object_list:
        object_img_list = os.listdir(object_path + "/" + one_object_folder)
        object_img_name = random.choice(object_img_list)
        
        object_img = cv2.imread(object_path +"/"+ one_object_folder +"/"+ object_img_name, cv2.IMREAD_UNCHANGED)
        
        class_num = object_img_name.split("_")[1][:-4]
    
        object_img_h = object_img.shape[0]
        object_img_w = object_img.shape[1]
        
        object_ratio = float(object_img_w) / float(object_img_h)
        
        if object_ratio > 1: # w > h
            resize_object_w = random.randrange(int(background_img_w * 0.2), int(background_img_w * 0.6))
            resize_object_h = int(resize_object_w * (1 / object_ratio))

        else:
            resize_object_h = random.randrange(int(background_img_h * 0.2), int(background_img_h * 0.6))
            resize_object_w = int(resize_object_h * object_ratio)

        # random object (x,y)
        random_x = random.randrange(0, int(background_img_w * 0.7))
        random_y = random.randrange(0, int(background_img_h * 0.7))

        if random_x + resize_object_w > background_img_w:
            resize_object_w = background_img_w - random_x

        if random_y + resize_object_h > background_img_h:
            resize_object_h = background_img_h - random_y


        label_w_ratio = float(resize_object_w) / float(background_img_w)
        label_h_ratio = float(resize_object_h) / float(background_img_h)

#         label_w_ratio = float(resize_object_w) / float(background_img_w)
#         label_h_ratio = float(resize_object_h) / float(background_img_h)

        # label x ratio & y ratio
        label_x_ratio = float(random_x + resize_object_w/2) / float(background_img_w)
        label_y_ratio = float(random_y + resize_object_h/2) / float(background_img_h)
        
        #print(resize_object_w, resize_object_h)
        # object resize and overlay to background
        resize_object_img = cv2.resize(object_img,(resize_object_w,resize_object_h))
        background_img = overlay_transparent(background_img, resize_object_img, random_x, random_y)

        # label txt write
        label_txt_file.write(str(class_num)+" "+\
                             str(label_x_ratio)+" "+\
                             str(label_y_ratio)+" "+\
                             str(label_w_ratio)+" "+\
                             str(label_h_ratio)+"\n")
    
    cv2.imwrite(overlay_path+"/"+str(cnt)+".jpg", background_img)
    label_txt_file.close()
    cnt += 1

8100


IOError: [Errno 28] No space left on device